In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
compare_loss={}
compare_accuracy={}

Using TensorFlow backend.


In [2]:
# function to convert image to array
def read_img(address, size):
    '''Read and resize image.
    Return image as numpy array, by normalizing the values'''
    img = image.load_img(address, target_size = size)
    img = image.img_to_array(img)
    return img
    
# function to convert labels to one hot encoding vector
def OneHotEncoded(y_train):
    y_t=np.zeros((len(y_train),Num_Class), dtype=int)
    for i,x in enumerate(y_train):
        #y_t[i][int(x)-1]=1
        y_t[i][int(x)] = 1
    return y_t


In [3]:
import os
import csv
from shutil import copy2
INPUT_SIZE = 224
directory_lists=os.listdir("./imgs/train")
labels = {}
data = {}

with open('./labels.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id','class'])
    for i in range(0, len(directory_lists)):                        #'''directory_lists == files in /train == ['c0', 'c1', ... , 'c9']'''
        t = directory_lists[i][1:]
        imag=os.listdir("./imgs/train/"+str(directory_lists[i]))     #'''imag == files in /train/c# '''
        current_loc = "./imgs/train/"+str(directory_lists[i])+"/"    #'''./imgs/train/c#/'''
        for k in range(0, len(imag)):
            x=[current_loc+str(imag[k])]+[t]
            writer.writerow(x)

data_dir = './'
labels = pd.read_csv(join(data_dir, 'labels.csv'))

INPUT_SIZE = 224
POOLING = 'avg'
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
y_train= np.zeros((len(labels),), dtype='float32')

for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
    y_train[i]=int(labels['class'][i])
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

Num_Class=10  # number of classes in dataset
# convert labels to one hot encoding
y_train=OneHotEncoded(y_train)
print(y_train)
# split data in to training and validation set
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.33, random_state=42)


22424it [02:20, 159.35it/s]


Train Images shape: (22424, 224, 224, 3) size: 3,375,439,872
[[1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]]


In [4]:
import h5py
import numpy as np

train_vgg_bf = np.load('train_vgg_bf.npy')
train_x_bf = np.load('train_x_bf.npy')
train_i_bf = np.load('train_i_bf.npy')

valid_vgg_bf = np.load('valid_vgg_bf.npy')
valid_x_bf = np.load('valid_x_bf.npy')
valid_i_bf = np.load('valid_i_bf.npy')


In [ ]:
'''import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)'''

In [11]:
X = np.hstack([train_x_bf, train_vgg_bf, train_i_bf ])
V = np.hstack([valid_x_bf, valid_vgg_bf, valid_i_bf])
print('Full train bottleneck features shape: {} size: {:,}'.format(X.shape, X.size))
print('Full valid bottleneck features shape: {} size: {:,}'.format(V.shape, V.size))

Full train bottleneck features shape: (15024, 4608) size: 69,230,592
Full valid bottleneck features shape: (7400, 4608) size: 34,099,200


In [12]:
from sklearn.linear_model import LogisticRegression
Num_Class=10

logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=1987)
logreg.fit(X, (y_train * range(Num_Class)).sum(axis=1))
valid_probs = logreg.predict_proba(V)
valid_preds = logreg.predict(V)
compare_loss['Xception_Inception']=log_loss(y_val, valid_probs)
compare_accuracy['Xception_Inception']=accuracy_score((y_val * range(Num_Class)).sum(axis=1), valid_preds)
print('Validation Xception + Inception LogLoss {}'.format(compare_loss['Xception_Inception']))
print('Validation Xception + Inception Accuracy {}'.format(compare_accuracy['Xception_Inception']))

Validation Xception + Inception LogLoss 0.0937344483617
Validation Xception + Inception Accuracy 0.981216216216


10
